In [2]:
import importData as import_data
import torch
import torch.nn as nn
import torch.optim as optim
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
from torch.utils.data import DataLoader, TensorDataset, random_split, Dataset
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
import numpy as np
import random
import copy
np.set_printoptions(edgeitems=10)
torch.backends.cuda.matmul.allow_tf32 = True

uuids = ["69508C29-2B15-4230-BE32-328E553EC63D", "2D2247C8-6B76-45BE-AC6B-F177A87ED4EF", "98006881-B2F0-4B6B-94CF-A21ABE69F21B", "94ABBF79-C4A7-4715-A661-94BC30627CBD", "DC6DB199-0AA7-441E-B896-C70A34CF7B6C", "2F6A5D84-2523-4EE2-BF3D-7693665B2258", "9026576A-3EA6-4A42-8A7A-0FD1290CF5C9", "BAB130C6-7D85-489A-B52E-CB0E149E585D"]
id = import_data.importData()
d_final, t_final = None, None
for uuid in uuids:
    d, t = id.import_data(uuid, 0.2, 100, 2)
    if d_final is None:
        d_final, t_final = d, t
    else:
        # print("o")
        if len(t_final[0]) == len(t[0]):
            d_final = np.concatenate((d_final, d), axis=0)
            t_final = np.concatenate((t_final, t), axis=0)
        else:
            if len(t_final[0]) < len(t[0]):
                difference = -len(t_final[0]) + len(t[0])
                lengthened_t_final = []
                for i, t_fin in enumerate(t_final):
                    # print(np.shape(t_fin))
                    zeros = np.zeros((difference,1))
                    # print(np.shape(zeros))
                    # print(t_fin)
                    # print(zeros)
                    lengthened_t_final.append(np.concatenate((t_fin, np.zeros((difference,1))), axis=0))
                d_final = np.concatenate((d_final, d), axis=0)
                t_final = np.concatenate((lengthened_t_final, t), axis=0)
            elif len(t_final[0]) > len(t[0]):
                difference = len(t_final[0]) - len(t[0])
                lengthened_t= []
                for i, t_fin in enumerate(t):
                    # print(np.shape(t_fin))
                    zeros = np.zeros((difference,1))
                    # print(np.shape(zeros))
                    # print(t_fin)
                    # print(zeros)
                    lengthened_t.append(np.concatenate((t_fin, np.zeros((difference,1))), axis=0))
                d_final = np.concatenate((d_final, d), axis=0)
                t_final = np.concatenate((t_final, lengthened_t), axis=0)
# print(t_final)             
t_final[t_final != 0] = 1

t_final = np.sum(t_final, axis=1)
# print(t_final)
t_final = np.squeeze(t_final, axis=-1)
# print(t_final)



testuuid = "99AAC7DE-4106-43AB-AA80-F5816AB4DB85"
d_test, t_test = id.import_data(testuuid, 0.2, 100, 2)
t_test_original = copy.deepcopy(t_test)
t_test[t_test != 0] = 1
t_test = np.sum(t_test, axis=1)
t_test = np.squeeze(t_test, axis=-1)
# indices = list(range(len(d_test)))
# sampled_indices = random.sample(indices, 1000)
# sampled_indices = sorted(sampled_indices)
# d_test, t_test = d_test[sampled_indices], t_test[sampled_indices]





/Users/thomas/Development/AATraining/importData.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed_raw_df["time"] = trimmed_raw_df["time"].apply(lambda x: x - timestamps_df["startTime"][0])
/Users/thomas/Development/AATraining/importData.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_time_dataset["time"] = start_time_dataset["time"] - start_time
/Users/thomas/Development/AATraining/importData.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [3]:
unique_t, counts = np.unique(t_final, return_counts=True)
print("Unique Elements: ", unique_t)
print("Counts: ", counts)

Unique Elements:  [0. 1.]
Counts:  [3593 1090]


In [7]:
class SitUpDataset(Dataset):
    def __init__(self, inputs, targets):
        """
        Args:
            inputs (numpy array): Input data of shape (num_examples, num_features, num_readings, 1)
            targets (numpy array): Target data of shape (num_examples, num_outputs, 1)
            max_outputs (int): Maximum number of outputs per example
        """
        self.inputs = torch.tensor(inputs, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)
        # self.max_outputs = max_outputs

        # Replace missing target times with -1 and create a mask
        # self.masks = (self.targets > 0).float()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        # Remove the last dimension of size 1
        x = self.inputs[idx].squeeze(-1)  # Shape: (num_features, num_readings)
        y = self.targets[idx]  # Shape: (num_outputs)
        # mask = self.masks[idx].squeeze(-1)  # Shape: (num_outputs)
        return x, y#, mask

class SitUpModel(nn.Module):
    def __init__(self, num_features, num_classes):
        super(SitUpModel, self).__init__()
        
        # Define the model layers
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=7, padding=2)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(128, num_classes)
        self.sm = nn.Softmax()
    
    def forward(self, x):
        # x shape: (batch_size, num_features, num_readings)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)  # x shape: (batch_size, 128, 1)
        x = x.view(x.size(0), -1)  # Flatten to (batch_size, 128)
        x = self.fc(x)  # Output shape: (batch_size, num_classes)
        x = self.sm(x)
        return x
    
def mse_loss(predictions, targets):
    # Compute MSE loss only over valid targets
    diff = (predictions - targets)
    loss = (diff ** 2).sum() / len(predictions)
    return loss



# Set device
if   torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

x_test = torch.tensor(d_test, dtype=torch.float32).to(device)
test_dataset = SitUpDataset(x_test, t_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Split Data into validation and training sets
X_train, X_val, Y_train, Y_val = train_test_split(d_final, t_final, test_size=0.01)
print(np.shape(X_train))

# Gather information on the dataset
num_classes = int(np.max(Y_train, axis=0) + 1)
num_features = len(X_train[0])
print(f"number of classes: {num_classes}")
print(f"number of features: {num_features}")

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
Y_train = torch.tensor(Y_train, dtype=torch.float32).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
Y_val = torch.tensor(Y_val, dtype=torch.float32).to(device)

# Create datasets and dataloaders
train_dataset = SitUpDataset(X_train, Y_train)
val_dataset = SitUpDataset(X_val, Y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

# Initialize the model, optimizer, and scheduler with new learning rate
model = SitUpModel(num_features=num_features, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

# Training loop
num_epochs = 50  # Initial number of epochs
for epoch in range(num_epochs):
    # Training
    model.train()
    total_train_loss = 0.0
    total_train_correct = 0
    for x_batch, y_batch in train_loader:
        # Load the batch and conver to the correct datatypes
        x_batch = x_batch.to(device)
        y_batch = y_batch.long().to(device)

        # Calculate batch outputs and loss
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)

        # Optimise for the next step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculate training stats
        total_train_loss += loss.item() * x_batch.size(0)
        _, predicted = torch.max(outputs, 1)
        total_train_correct += (predicted == y_batch).sum().item()

    # Validation
    model.eval()
    total_eval_loss = 0.0
    total_eval_correct = 0
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            # Load the batch and conver to the correct datatypes
            x_batch = x_batch.to(device)
            y_batch = y_batch.long().to(device)

            # Calculate batch outputs and loss
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            
            # Calculate training stats
            total_eval_loss += loss.item() * x_batch.size(0)
            _, predicted = torch.max(outputs, 1)
            total_eval_correct += (predicted == y_batch).sum().item()


    scheduler.step()
    avg_train_loss = total_train_loss / len(train_dataset)
    train_accuracy = 100* total_train_correct / len(train_dataset)
    avg_val_loss = total_eval_loss / len(test_dataset)
    val_accuracy = 100* total_eval_correct / len(test_dataset)

    if (epoch +1)% 5 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%, learning rate: {scheduler.get_last_lr()[0]:.6f}")

# Save the trained model and optimizer state
torch.save({
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'loss': loss.item(),
}, 'checkpoint.pth')

/var/folders/y8/qjgzy9v90vnc5zwq1l22vbvw0000gn/T/ipykernel_98460/962128042.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.inputs = torch.tensor(inputs, dtype=torch.float32)
/var/folders/y8/qjgzy9v90vnc5zwq1l22vbvw0000gn/T/ipykernel_98460/962128042.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.targets = torch.tensor(targets, dtype=torch.float32)


(4636, 9, 200, 1)
number of classes: 2
number of features: 9
Epoch 5/50, Train Loss: 0.3748, Train Accuracy: 94.50%, Val Loss: 0.4485, Val Accuracy: 88.46%, learning rate: 0.000500
Epoch 10/50, Train Loss: 0.3705, Train Accuracy: 95.17%, Val Loss: 0.4253, Val Accuracy: 92.31%, learning rate: 0.000250
Epoch 15/50, Train Loss: 0.3672, Train Accuracy: 95.38%, Val Loss: 0.4125, Val Accuracy: 91.94%, learning rate: 0.000125
Epoch 20/50, Train Loss: 0.3648, Train Accuracy: 95.82%, Val Loss: 0.4218, Val Accuracy: 89.74%, learning rate: 0.000063
Epoch 25/50, Train Loss: 0.3610, Train Accuracy: 96.18%, Val Loss: 0.4176, Val Accuracy: 91.76%, learning rate: 0.000031
Epoch 30/50, Train Loss: 0.3600, Train Accuracy: 96.31%, Val Loss: 0.4203, Val Accuracy: 91.03%, learning rate: 0.000016
Epoch 35/50, Train Loss: 0.3598, Train Accuracy: 96.31%, Val Loss: 0.4060, Val Accuracy: 91.94%, learning rate: 0.000008
Epoch 40/50, Train Loss: 0.3588, Train Accuracy: 96.53%, Val Loss: 0.4157, Val Accuracy: 91.7

In [8]:
import copy
testuuid = "94ABBF79-C4A7-4715-A661-94BC30627CBD"
d_test, t_test = id.import_data(testuuid, 0.2, 100, 2)
t_test_original = copy.deepcopy(t_test)
t_test[t_test != 0] = 1
t_test = np.sum(t_test, axis=1)
t_test = np.squeeze(t_test, axis=-1)
x_test = torch.tensor(d_test, dtype=torch.float32).to(device)
test_dataset = SitUpDataset(x_test, t_test)
test_loader = DataLoader(test_dataset, batch_size=64)
checkpoint = torch.load('checkpoint.pth')
        
# Load the saved model
model = SitUpModel(num_features, num_classes).to(device)
model.load_state_dict(checkpoint['model_state_dict'])

# Testing
model.eval()
total_test_loss = 0.0
total_test_correct = 0
batch_num = 0
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        # Load the batch and conver to the correct datatypes
        x_batch = x_batch.to(device)
        y_batch = y_batch.long().to(device)

        # Calculate batch outputs and loss
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        
        # Calculate training stats
        total_test_loss += loss.item() * x_batch.size(0)
        _, predicted = torch.max(outputs, 1)
        total_test_correct += (predicted == y_batch).sum().item()

        # Convert the outputs to a NumPy array
        outputs = outputs.cpu().numpy()

        for i, output in enumerate(outputs):
            output_num_situps = np.argmax(output)
            true_num_situps = y_batch[i]
            if true_num_situps == output_num_situps:
                verify_mark = "✅"
            else:
                verify_mark = "❌"
            print(verify_mark, output, y_batch[i].item(), t_test_original[batch_num*64 + i].flatten())
        batch_num += 1

avg_test_loss = total_test_loss / len(test_dataset)
test_accuracy = 100* total_test_correct / len(test_dataset)
print(f"Test loss: {avg_test_loss}, Test accuracy: {test_accuracy}")

# # Run the model on the new dataset
# with torch.no_grad():
#     outputs = model(test_data)
#     total_test_loss = loss.item() * x_batch.size(0)
#     _, predicted = torch.max(outputs, 1)
#     total_eval_correct += (predicted == y_batch).sum().item()







/Users/thomas/Development/AATraining/importData.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed_raw_df["time"] = trimmed_raw_df["time"].apply(lambda x: x - timestamps_df["startTime"][0])
/Users/thomas/Development/AATraining/importData.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_time_dataset["time"] = start_time_dataset["time"] - start_time
/var/folders/y8/qjgzy9v90vnc5zwq1l22vbvw0000gn/T/ipykernel_98460/962128042.py:9: UserWarning: To copy construct from a tensor, it is recomme

✅ [0.986584   0.01341602] 0 [0.]
✅ [0.98611754 0.01388246] 0 [0.]
✅ [0.9894449  0.01055506] 0 [0.]
✅ [0.984611   0.01538904] 0 [0.]
✅ [0.96502966 0.03497033] 0 [0.]
❌ [0.89380115 0.10619882] 1 [1.949934]
❌ [0.59882873 0.40117127] 1 [1.749934]
✅ [0.1592231 0.8407769] 1 [1.549934]
✅ [0.03039449 0.9696055 ] 1 [1.349934]
✅ [0.01066818 0.98933184] 1 [1.149934]
✅ [0.01414452 0.9858555 ] 1 [0.949934]
✅ [0.01518527 0.98481476] 1 [0.749934]
✅ [0.01194578 0.9880542 ] 1 [0.549934]
✅ [0.01794438 0.9820556 ] 1 [0.349934]
✅ [0.03752715 0.9624728 ] 1 [0.14993401]
❌ [0.11384608 0.88615394] 0 [0.]
❌ [0.37296373 0.6270363 ] 0 [0.]
✅ [0.7132903  0.28670976] 0 [0.]
✅ [0.90723324 0.09276675] 0 [0.]
✅ [0.9690494  0.03095059] 0 [0.]
✅ [0.9865975  0.01340253] 0 [0.]
✅ [0.9929584  0.00704155] 0 [0.]
✅ [0.99614394 0.003856  ] 0 [0.]
✅ [0.99748635 0.00251359] 0 [0.]
✅ [0.99747247 0.00252748] 0 [0.]
✅ [0.99769694 0.00230305] 0 [0.]
✅ [0.99336195 0.00663807] 0 [0.]
✅ [0.9820811  0.01791885] 0 [0.]
✅ [0.945192   0.

In [ ]:
# # Load the checkpoint
# checkpoint = torch.load('checkpoint.pth')

# # Reinitialize the model and optimizer
# model = SitUpModel(num_features=num_features, num_classes=num_classes).to(device)
# optimizer = torch.optim.Adam(model.parameters())
# scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

# # Load the state dictionaries
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

# # Retrieve the last epoch
# start_epoch = checkpoint['epoch']
# loss = checkpoint['loss']

# # Define the number of additional epochs
# additional_epochs = 300
# total_epochs = start_epoch + additional_epochs

# # Training loop  # Initial number of epochs
# for epoch in range(start_epoch, total_epochs):
#     model.train()
#     total_loss = 0.0
#     correct = 0
#     for x_batch, y_batch in dataloader:
#         x_batch = x_batch.permute(0, 1, 2).to(device)
#         y_batch = y_batch.long().to(device)
#         # mask_batch = mask_batch.to(device)

#         outputs = model(x_batch)
#         # print(outputs)
#         # print(y_batch)
#         loss = criterion(outputs, y_batch)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        

#         total_loss += loss.item() * x_batch.size(0)
#         _, predicted = torch.max(outputs, 1)
#         correct += (predicted == y_batch).sum().item()

        

#     scheduler.step()
#     avg_loss = total_loss / len(dataset)
#     accuracy = 100* correct / len(dataset)

#     print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%, learning rate: {scheduler.get_last_lr()[0]:.6f}")
#         # print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Learning Rate: {scheduler.get_last_lr()[0]:.6f}")

# # for epoch in range(start_epoch, total_epochs):
# #     model.train()
# #     for x_batch, y_batch in dataloader:
# #         x_batch = x_batch.permute(0, 1, 2).to(device)
# #         y_batch = y_batch.to(device)
# #         # mask_batch = mask_batch.to(device)

# #         predictions = model(x_batch)
# #         loss = mse_loss(predictions, y_batch)
# #         optimizer.zero_grad()
# #         loss.backward()
# #         optimizer.step()

# #     scheduler.step()

# #     print(f"Epoch {epoch + 1}/{total_epochs}, Loss: {loss.item():.4f}, Learning Rate: {scheduler.get_last_lr()[0]:.6f}")

# # Save the model again after additional training
# torch.save({
#     'epoch': total_epochs,
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(),
#     'scheduler_state_dict': scheduler.state_dict(),
#     'loss': loss.item(),
# }, 'checkpoint.pth')

/tmp/ipykernel_52082/3400682453.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('checkpoint.pth')


NameError: name 'dataloader' is not defined